In [14]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define data transformations for training and validation datasets
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (required by EfficientNet)
    transforms.ToTensor(),  # Convert image to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

# Load the training and validation datasets
train_dataset = datasets.ImageFolder('Training', transform=transform)
val_dataset = datasets.ImageFolder('Validation', transform=transform)

# Create DataLoader instances for both training and validation data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [10]:
# Check if GPU is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [15]:
from efficientnet_pytorch import EfficientNet
import torch.nn as nn

# Load EfficientNet-B0 pre-trained model
model = EfficientNet.from_pretrained('efficientnet-b0')

# Modify the final fully connected layer to match the number of classes in your dataset
model._fc = nn.Linear(model._fc.in_features, 4)  # Assuming 4 classes in your dataset

# Move the model to the chosen device (GPU or CPU)
model = model.to(device)


Loaded pretrained weights for efficientnet-b0


In [16]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Common choice for classification problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Number of epochs for training
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize the model parameters

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


Epoch 1/10, Loss: 0.0884, Accuracy: 97.24%
Epoch 2/10, Loss: 0.0413, Accuracy: 98.81%


In [ ]:
# Evaluation on Validation Data
model.eval()  # Set the model to evaluation mode
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # No need to compute gradients for validation
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss = val_loss / len(val_loader)
val_acc = correct / total * 100
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")
